In [19]:
from collections import defaultdict

from notebooks.rewrite import edge_index
from src.encoding.types import VSAModel
import torch
import torchhd

from tests_new.test_torchhd_utils import cartesian_bind_tensor

l = list(tuple(VSAModel))

vsa = VSAModel.MAP
a, b = torchhd.random(2, 8, vsa=vsa.value)
print(f"{a=}")
print(f"{b=}")

a_b = torch.stack([a, b]).squeeze()
print(f"{a_b=}")

a=MAPTensor([-1.,  1., -1.,  1.,  1.,  1.,  1., -1.])
b=MAPTensor([ 1.,  1., -1., -1., -1.,  1.,  1., -1.])
a_b=MAPTensor([[-1.,  1., -1.,  1.,  1.,  1.,  1., -1.],
           [ 1.,  1., -1., -1., -1.,  1.,  1., -1.]])


In [ ]:
print(f"{torchhd.bind(a, b)=}")
print(f"{torchhd.multibind(a_b)=}")
print(f"{torch.equal(torchhd.bind(a, b), torchhd.multibind(a_b))=}")

In [ ]:
print(f"{torchhd.bundle(a, b)=}")
print(f"{torchhd.multibundle(a_b)=}")
print(f"{torch.equal(torchhd.bundle(a, b), torchhd.multibundle(a_b))=}")

In [ ]:
print(f"{a_b.shape=}")
a_b = torch.unsqueeze(a_b, dim=0)
print(f"{a_b.shape=}")
a_b_x_2 = torch.cat([a_b, a_b], dim=0)
print(f"{a_b_x_2.shape=}")

In [ ]:
multibundle = torchhd.multibundle(a_b_x_2)
print(f"{ multibundle=}")

In [ ]:
import torch
import torchhd

R = torchhd.random(5, 10)
print(f"{R.shape=}")
G = torchhd.random(5, 10)
print(f"{G.shape=}")

RG = torch.stack([R, G], dim=0)  # [2, 5, 10]
print(f"{RG.shape}")

## We want to multibind on the 0th dimension
RGT = RG.transpose(0, 1)
print(f"{RGT.shape=}")

y = torchhd.multibind(RGT)
print(f"{y.shape=}")

In [ ]:
import torch
import torchhd
from torchhd import functional as F, HRRTensor  # or just use torchhd.bind

# your random hypervectors
R = torchhd.random(5, 10)
G = torchhd.random(5, 10)
B = torchhd.random(5, 10)

# stack & multibind via transpose (what you already did)
RG = torch.stack([R, G, B], dim=0)  # [2, 5, 10]
y = torchhd.multibind(RG.transpose(0, 1))  # → [5, 10]

# manual two-way bind along N
# torchhd.bind (alias for the MAP-model binding op) does elementwise bind of two HVs
y2 = torchhd.bind(R, G).bind(
    B
)  # → [5, 10]  [oai_citation:0‡Torchhd](https://torchhd.readthedocs.io/en/stable/torchhd.html)

# check they’re identical
print("shapes:", y.shape, y2.shape)
print("equal?:", torch.equal(y, y2))  # → True

l = [R, G, B]
# start from the first element...
y3 = l[0]
# ...then iteratively bind in the rest
for hv in l[1:]:
    y3 = torchhd.bind(y3, hv)

print("shapes:", y.shape, y3.shape)  # both torch.Size([5, 10])
print("equal?:", torch.equal(y, y3))  # → True

## True False Hyper vectors

In [ ]:
import torchhd
from torchhd import functional as F

D = 10
true_hv = torchhd.random(1, D)[0]  # e.g. shape [D]
false_hv = F.inverse(true_hv)  # inverse for binding

# check: binding true⊗false → identity
I = torchhd.identity(1, D)[0]
print(torch.equal(torchhd.bind(true_hv, false_hv), I))

## Create Torchhd BSC Tensor Manually - Bundle vs. Multibundle Problem

In [ ]:
from torchhd import BSCTensor

# 1) as a plain Boolean torch.Tensor
a = torch.tensor([True, True], dtype=torch.bool)
a_hv = BSCTensor(a)
z = a_hv.new_zeros(size=a_hv.size())
b = torch.tensor([False, False], dtype=torch.bool)
b_hv = BSCTensor(b)

## This is a strange behaviour, the majority voting tie-break of multi bundle and bundle do not result in the same vector
res1 = a_hv.bundle(b_hv)  # False, False
res2 = torchhd.multibundle(torch.stack([a_hv, b_hv]))  # True, False
print(f"{torch.equal(res1, res2)=}")

## Create Torchhd MAP Tensor Manually - Bundle vs. Multibundle Problem

In [ ]:
import torch
import torchhd
from torchhd.tensors.map import MAPTensor

# Fix a small dimension for demo
D = 2
torch.manual_seed(0)

# Create two random MAP hypervectors of shape [2]
a_hv = MAPTensor.random(1, D).squeeze(0)
b_hv = MAPTensor.random(1, D).squeeze(0)

# 1) Two-way bundle (elementwise sum)
res1 = torchhd.bundle(a_hv, b_hv)

# 2) Multibundle (sum across the first dim)
res2 = torchhd.multibundle(torch.stack([a_hv, b_hv], dim=0))

print("a_hv:", a_hv)
print("b_hv:", b_hv)
print("res1 (bundle):   ", res1)
print("res2 (multibundle):", res2)
print("Equal?:", torch.equal(res1, res2))  # → True

## Multibind, Mulitbundle

In [1]:
import torch
import torchhd
from torchhd.tensors.hrr import HRRTensor

# Fix a small dimension for demo
D = 5
torch.manual_seed(0)

# Create two random MAP hypervectors of shape [2]
a = HRRTensor.random(1, D).squeeze(0)
a_bind = torchhd.multibind(torch.stack([a]))
a_bind_b = torchhd.bind(a, a)
a_bundle = torchhd.multibundle(torch.stack([a]))
a_bundle_b = torchhd.bundle(a, a)

print("a:", a)
print("a_bind:", a_bind)
print("Equal?:", torch.equal(a, a_bind))  # → True
print("a_bind_b", a_bind_b)

print("a_bundle:", a_bundle)
print("Equal?:", torch.equal(a, a_bind))  # → False
print("a_bundle_b", a_bundle_b)

# Important: the multibind and multibundle apply a self-bundle/bind if on a single input. So you should always check the dimensions, and not apply the function if the dim(-2) is one. But it seems like bind(a, a) is not equal multibind(a). Whatos going on?

a: HRRTensor([ 0.5222, -0.0994, -0.7384,  0.1926, -0.3675])
a_bind: HRRTensor([ 0.5222, -0.2335, -0.2729, -0.2729, -0.2335])
Equal?: False
a_bind_b HRRTensor([ 0.0613,  0.4760, -0.9029,  0.4831,  0.1230])
a_bundle: HRRTensor([ 0.5222, -0.0994, -0.7384,  0.1926, -0.3675])
Equal?: False
a_bundle_b HRRTensor([ 1.0445, -0.1989, -1.4767,  0.3853, -0.7351])


/Users/arvandkaveh/Projects/kit/graph_hdc/.venv/lib/python3.13/site-packages/torch/_tensor.py:1668: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:307.)
  ret = func(*args, **kwargs)


In [35]:
import torch
import torchhd


# Fix a small dimension for demo
D = 5
torch.manual_seed(0)


a = torchhd.random(1, D, vsa="HRR")
b = torchhd.random(1, D, vsa="HRR")

a = a.squeeze(0)
b = b.squeeze(0)

# normal binds
a_b = torchhd.bind(a, a).bind(b)
print("a_b:", a_b)

# multibind
s = torch.stack([a, b], dim=0)
a_b_m = torchhd.multibind(s)
print("a_b_m:", a_b_m)

assert torch.allclose(a_b, a_b_m)

a_b: HRRTensor([ 0.4415, -0.2786,  0.6536, -0.3927, -0.5884])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

## Test multiset multisetity

In [33]:
import torch
import torchhd
from src.encoding.configs_and_constants import VSAModel

#### HRR
# Fix a small dimension for demo
D = 256
vsa = VSAModel.HRR
# Create two random MAP hypervectors of shape [2]
fruits = torchhd.random(16, D, vsa=vsa.value)
books = torchhd.random(10, D, vsa=vsa.value)

f = 0
b = 0
bundle = torchhd.multibundle(
    torch.stack(
        [
            fruits[f],
            fruits[f],
            # fruits[f],
            # fruits[f],
            # fruits[f],
            books[b],
            # books[b],
            # books[b],
        ]
    )
)

simf = torch.round(torchhd.dot(bundle, fruits)).clamp(min=0)
simb = torch.round(torchhd.dot(bundle, books)).clamp(min=0)

print(f"###############{repr(vsa)}")
print(simf)
print(simb)
f_idx = simf.argmax().item()
print(f"Index is Correct: {(f ==f_idx)=}")
print(f"Count is correct: {simf[f_idx]=}, {(simf[f_idx]==5)=}")
b_idx = simb.argmax().item()
print(f"Index is Correct: {(b ==b_idx)=}")
print(f"Count is correct: {simb[b_idx]=}, {(simb[b_idx]==3)=}")
print("\n--------------------------------\n")


#### FHRR
# Fix a small dimension for demo
vsa = VSAModel.FHRR
# Create two random MAP hypervectors of shape [2]
fruits = torchhd.random(16, D, vsa=vsa.value)
books = torchhd.random(10, D, vsa=vsa.value)

bundle = torchhd.multibundle(
    torch.stack(
        [
            fruits[f],
            fruits[f],
            # fruits[f],
            # fruits[f],
            # fruits[f],
            books[b],
            # books[b],
            # books[b],
        ]
    )
)

simf = (torchhd.dot(bundle, fruits) / D).round().int().clamp(min=0)
simb = torch.round(torchhd.dot(bundle, books) / D).clamp(min=0)

print(f"###############{repr(vsa)}")
print(simf)
print(simb)
f_idx = simf.argmax().item()
print(f"Index is Correct: {(f ==f_idx)=}")
print(f"Count is correct: {simf[f_idx]=}, {(simf[f_idx]==5)=}")
b_idx = simb.argmax().item()
print(f"Index is Correct: {(b ==b_idx)=}")
print(f"Count is correct: {simb[b_idx]=}, {(simb[b_idx]==3)=}")
print("\n--------------------------------\n")

#### FHRR
# Fix a small dimension for demo
vsa = VSAModel.MAP
# Create two random MAP hypervectors of shape [2]
fruits = torchhd.random(16, D, vsa=vsa.value)
books = torchhd.random(10, D, vsa=vsa.value)

bundle = torchhd.multibundle(
    torch.stack(
        [
            fruits[f],
            fruits[f],
            # fruits[f],
            # fruits[f],
            # fruits[f],
            books[b],
            # books[b],
            # books[b],
        ]
    )
)

simf = (torchhd.dot(bundle, fruits) / D).round().int().clamp(min=0)
simb = (torchhd.dot(bundle, books) / D).round().int().clamp(min=0)

print(f"###############{repr(vsa)}")
print(simf)
print(simb)
f_idx = simf.argmax().item()
print(f"Index is Correct: {(f ==f_idx)=}")
print(f"Count is correct: {simf[f_idx]=}, {(simf[f_idx]==5)=}")
b_idx = simb.argmax().item()
print(f"Index is Correct: {(b ==b_idx)=}")
print(f"Count is correct: {simb[b_idx]=}, {(simb[b_idx]==3)=}")
print("\n--------------------------------\n")


#### VTB
# Fix a small dimension for demo
vsa = VSAModel.VTB
# Create two random MAP hypervectors of shape [2]
fruits = torchhd.random(16, D, vsa=vsa.value)
books = torchhd.random(10, D, vsa=vsa.value)

bundle = torchhd.multibundle(
    torch.stack(
        [
            fruits[f],
            fruits[f],
            # fruits[f],
            # fruits[f],
            # fruits[f],
            books[b],
            # books[b],
            # books[b],
        ]
    )
)

simf = (torchhd.dot(bundle, fruits)).round().int().clamp(min=0)
simb = (torchhd.dot(bundle, books)).round().int().clamp(min=0)

print(f"###############{repr(vsa)}")
print(simf)
print(simb)
f_idx = simf.argmax().item()
print(f"Index is Correct: {(f ==f_idx)=}")
print(f"Count is correct: {simf[f_idx]=}, {(simf[f_idx]==5)=}")
b_idx = simb.argmax().item()
print(f"Index is Correct: {(b ==b_idx)=}")
print(f"Count is correct: {simb[b_idx]=}, {(simb[b_idx]==3)=}")
print("\n--------------------------------\n")

###############<VSAModel.HRR: 'HRR'>
HRRTensor([2., 0., 0., -0., 0., -0., 0., -0., -0., 0., 0., -0., 0., 0., -0., 0.])
HRRTensor([1., -0., 0., -0., -0., 0., -0., 0., 0., 0.])
Index is Correct: (f ==f_idx)=True
Count is correct: simf[f_idx]=HRRTensor(2.), (simf[f_idx]==5)=HRRTensor(False)
Index is Correct: (b ==b_idx)=True
Count is correct: simb[b_idx]=HRRTensor(1.), (simb[b_idx]==3)=HRRTensor(False)

--------------------------------

###############<VSAModel.FHRR: 'FHRR'>
FHRRTensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
FHRRTensor([1., 0., -0., -0., 0., -0., -0., -0., 0., -0.])
Index is Correct: (f ==f_idx)=True
Count is correct: simf[f_idx]=FHRRTensor(2, dtype=torch.int32), (simf[f_idx]==5)=FHRRTensor(False)
Index is Correct: (b ==b_idx)=True
Count is correct: simb[b_idx]=FHRRTensor(1.), (simb[b_idx]==3)=FHRRTensor(False)

--------------------------------

###############<VSAModel.MAP: 'MAP'>
MAPTensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dt

## Dtypes

In [2]:
from src.encoding.configs_and_constants import VSAModel
import torchhd

for vsa in VSAModel:
    t = torchhd.random(1, 1, vsa=vsa.value)
    print(f"vsa:{vsa.value}, t.dtype:{t.dtype}")

vsa:MAP, t.dtype:torch.float32
vsa:HRR, t.dtype:torch.float32
vsa:FHRR, t.dtype:torch.complex64
vsa:VTB, t.dtype:torch.float32


## Multiple Node Feature Decoding

In [61]:
import torch
import torchhd
from src.encoding.configs_and_constants import VSAModel
from src.utils.utils import cartesian_bind_tensor
from collections import defaultdict

vsa = VSAModel.HRR
D = 1000
fruits = torchhd.random(10, D, vsa=vsa.value)
veggies = torchhd.random(6, D, vsa=vsa.value)

nodes = [
    torchhd.bind(fruits[0], veggies[0]),
    torchhd.bind(fruits[0], veggies[0]),
    torchhd.bind(fruits[0], veggies[0]),
    torchhd.bind(fruits[0], veggies[2]),
    torchhd.bind(fruits[0], veggies[4]),
    torchhd.bind(fruits[1], veggies[0]),
    torchhd.bind(fruits[1], veggies[0]),
    torchhd.bind(fruits[1], veggies[0]),
    torchhd.bind(fruits[1], veggies[2]),
    torchhd.bind(fruits[1], veggies[4]),
    torchhd.bind(fruits[2], veggies[0]),
    torchhd.bind(fruits[2], veggies[0]),
    torchhd.bind(fruits[2], veggies[0]),
    torchhd.bind(fruits[2], veggies[2]),
    torchhd.bind(fruits[2], veggies[4]),
    torchhd.bind(fruits[3], veggies[0]),
    torchhd.bind(fruits[3], veggies[0]),
    torchhd.bind(fruits[3], veggies[0]),
    torchhd.bind(fruits[3], veggies[2]),
    torchhd.bind(fruits[3], veggies[4]),
]
embedding_0 = torchhd.multiset(torch.stack(nodes, dim=0))

node_codebook = cartesian_bind_tensor([fruits, veggies])

sim_node = torchhd.dot(embedding_0, node_codebook).round().int().clamp(min=0)
## This should print 3, 0, 1, 0, 1, 0, 3, .... so many time as there items.
## And it does, so it should work
print(sim_node)


## Now level 1
edge_idx = list(zip(range(len(nodes)), range(len(nodes))))

edge_dict = defaultdict(list)
for src, dst in edge_idx:
    edge_dict[src].append(dst)
print(edge_dict)

edge_term_bindings = [torchhd.bind(nodes[src], torchhd.multiset(torch.stack([nodes[dst] for dst in dsts], dim=0))) for src, dsts in edge_dict.items()]
edge_terms = torchhd.multiset(torch.stack(edge_term_bindings, dim=0))
embedding_1 = torchhd.multiset(torch.stack([embedding_0, edge_terms], dim=0))

edge_codebook = cartesian_bind_tensor([node_codebook, node_codebook])

sim_edge = torchhd.dot(embedding_1, edge_terms).round().int().clamp(min=0)
## This should print 3, 0, 1, 0, 1, 0, 3, .... so many time as there items.
## And it does, so it should work
print(sim_edge)
print(sim_edge.sum().item())
print(len(nodes)*2)

HRRTensor([3, 0, 1, 0, 1, 0, 4, 0, 2, 0, 1, 0, 3, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
defaultdict(<class 'list'>, {0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19]})
HRRTensor(167, dtype=torch.int32)
167
40
